In [285]:
import requests
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time
from math import ceil
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [98]:
url = 'https://www.seafoodsource.com/supplier-directory/'
response = requests.get(url)
html = response.text

In [216]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--headless")
driver = uc.Chrome(options=options)
driver.get(url)

In [217]:
# Wait page to load
time.sleep(15)

In [218]:
# Get selection element for country
select_country = driver.find_element(By.CLASS_NAME, 'location-select')
select_country = select_country.find_element(By.TAG_NAME, 'select')

In [219]:
select = Select(select_country)

In [172]:
# Function to return the number of page by the amount of results
def get_pages_number(soup):
    results_div = soup.find('div', {'class': 'article-count'})
    results = int(results_div.text.replace(' results found.', '').replace(' ', ''))
    pages = ceil(results/16)

    return pages

In [220]:
supplier_url = []

In [221]:
# Gather all links for every supplier
for i in range(1,len(select.options)):
    
    # Select country
    select.select_by_index(i)
    time.sleep(0.7)

    # Gather page code from Selenium to BS
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Get number of pages
    pages = get_pages_number(soup)

    # Loop to go by every page
    for i in range(pages):
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        
        links = soup.find_all('a', {'class': 'btn btn-primary dc-view-listing'})

        # Collect all suppliers page links
        for i in links:
            supplier_url.append('https://www.seafoodsource.com' + i.get('href'))
    
        # Go to next page
        if pages > 1:
            driver.find_element(By.CLASS_NAME, 'ais-Pagination-item--nextPage').click()


In [222]:
len(supplier_url)

4294

In [293]:
df = pd.DataFrame(columns=['supplier_url', 'supplier_name', 'supplier_address_street',
       'supplier_address_city', 'supplier_address_state',
       'supplier_address_zip', 'supplier_address_country', 'categories',
       'events', 'supplier_web_address', 'supplier_email', 'supplier_phone',
       'supplier_fax'])

In [309]:
# With suppliers link, loop through them and get the data for each one of supplier
# Save each one into a dataframe
for i in supplier_url:
    # Request page
    url = i
    response = requests.get(url)
    html = response.text
    soup2 = BeautifulSoup(html, 'html.parser')

    # Collect data
    supplier_name = soup2.find('h1').text
    supplier_address_street = soup2.find_all('div', {'class': 'supplier-directory__address--street'})[0].text
    supplier_address_city = soup2.find_all('div', {'class': 'supplier-directory__address--city'})[0].text
    supplier_address_state = soup2.find_all('div', {'class': 'supplier-directory__address--state'})[0].text
    supplier_address_zip = soup2.find_all('div', {'class': 'supplier-directory__address--zip'})[0].text
    supplier_address_country = soup2.find_all('div', {'class': 'supplier-directory__address--country'})[0].text

    categories = soup2.find('div', {'class': 'supplier-directory__categories'}).find_all('li')
    categories_list = [i.text for i in categories]

    events = soup2.find('div', {'class': 'supplier-directory__events-associations'}).text.split(',')

    supplier_web_address = soup2.find('div', {'class': 'supplier-directory__web-address'}).text
    supplier_email = soup2.find('div', {'class': 'supplier-directory__email'}).text
    supplier_phone = soup2.find('div', {'class': 'supplier-directory__phone'}).text
    supplier_fax = soup2.find('div', {'class': 'supplier-directory__fax'}).text

    # Store data
    data = {'supplier_url': i, 
        'supplier_name': supplier_name, 
        'supplier_address_street': supplier_address_street,
        'supplier_address_city': supplier_address_city,
        'supplier_address_state': supplier_address_state,
        'supplier_address_zip': supplier_address_zip,
        'supplier_address_country': supplier_address_country,
        'categories': str(categories_list),
        'events': str(events),
        'supplier_web_address': supplier_web_address,
        'supplier_email': supplier_email,
        'supplier_phone': supplier_phone,
        'supplier_fax': supplier_fax}
    
    dataframe = pd.DataFrame(data=data, index=[0])

    result = [df, dataframe]
    df = pd.concat(result)
    time.sleep(0.5)

In [310]:
df

,supplier_url,supplier_name,supplier_address_street,supplier_address_city,supplier_address_state,supplier_address_zip,supplier_address_country,categories,events,supplier_web_address,supplier_email,supplier_phone,supplier_fax
0,https://www.seafoodsource.com/supplier-directo...,SFT Angola,"Torre Ambiente, Rua Major Kanhangulo No 11, Es...",Luanda,,,Angola,"['Fin Fish » Horse Mackerel', 'Shellfish » Shr...","['Seafood Expo Global 2023', ' Seafood Expo Gl...",,,+37046246661,
0,https://www.seafoodsource.com/supplier-directo...,DUNTOWER CORPORATION LIMITED,Castañeda 1871,Ciudad Autónoma de Buenos Aires,Buenos Aires,C1428ACT,Argentina,"['Fin Fish » Hake', 'Fin Fish » Sea Bass', 'Fi...","['Seafood Expo Global 2024', ' Seafood Expo Gl...",http://www.duntower.com,[email protected],+54.11.4788-0584,
0,https://www.seafoodsource.com/supplier-directo...,ICELAND SEAFOOD - Achernar S.A.S.,C/ Colombia 1525,Puerto Madryn,Chubut,09120,Argentina,"['Other Seafood » Roe', 'Fin Fish » Saithe - A...",['Seafood Expo North America 2023'],,,+54 2804456499,
0,https://www.seafoodsource.com/supplier-directo...,Bellini Export,MARCELO T DE ALVEAR 1045,"Mar del Plata, Buenos Aires",,07600,Argentina,[],['Seafood Expo Global 2024'],,,,
0,https://www.seafoodsource.com/supplier-directo...,Nedar S.A,"Av. Roque Saenz Peña 846, floor 7, office B",Ciudad de Buenos Aires,,1035,Argentina,[],['Seafood Expo Asia 2023'],http://www.videmar.es,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,https://www.seafoodsource.com/supplier-directo...,Tamimi Fisheries Company,"Ash Shihr, Hadramout","Ash Shihr, Hadramout",Hadramout,60079,Yemen,"['Fin Fish » Mahi Mahi', 'Fin Fish » Tuna', 'F...","['Seafood Expo Global 2024', ' Seafood Expo As...",,,00-967-5335492,
0,https://www.seafoodsource.com/supplier-directo...,LOBSTER SEAFOOD COMPANY LTD.,Lobster for exporting fish and sea food ltd.,"Ash-Shihr, Hadhramaut",,,Yemen,"['General Category » Frozen Seafood', 'Fin Fis...",['Seafood Expo Asia 2023'],http://www.lobsterseafood-co.com,,66-651-466-659,
0,https://www.seafoodsource.com/supplier-directo...,ADEN GULF FISHERIES CO. LTD.,P.O. BOX 4720,ADEN,ADEN,967,Yemen,"['Fin Fish » Mahi Mahi', 'Fin Fish » Catfish',...","['Seafood Expo Global 2023', ' Seafood Expo Gl...",,,967-239-1395,
0,https://www.seafoodsource.com/supplier-directo...,Motamaiz For Fish Export,"ASH SHIHR, HADRAMOUT,YEMEN",ASH SHIHR,HADHRAMOUT,,Yemen,"['Fin Fish » Grouper', 'Fin Fish » Mahi Mahi',...","['Seafood Expo Asia 2023', ' Seafood Expo Glob...",https://motamaizforfish.com/,,07-330-067-79,


In [311]:
df.to_csv('seafoodsource.csv')